<a href="https://colab.research.google.com/github/RUMONMD89/Computing-science-with-Fortran/blob/main/ACS7REV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/FORTRAN

Mounted at /content/drive
/content/drive/MyDrive/FORTRAN


In [2]:
%%writefile ACS7.f03
! _/_/_/ Gaussian Elimination with LU Decomposition _/_/_/
! 2022.08.19 Written by Y.Hirokawa
! Refference Code: https://ww2.odu.edu/~agodunov/computing/programs/book2/Ch06/Gauss_2.f90
! Refference     : https://qiita.com/kenjihiranabe/items/11d07d3cea0325f17851

program main
  implicit none
  integer, parameter :: NDIM=3, IDEBUG=1
  double precision   :: a(NDIM,NDIM), b(NDIM), d(NDIM,NDIM), x(NDIM)
  integer            :: i, j

  ! Initialize
  call sub_init(a,b,d,x,NDIM,IDEBUG)

  ! Output Data
  write(*,*) "=== Initial Value ==="
  call sub_output(a,b,d,x,NDIM)

  ! Gaussian Elimination
  call sub_gaussian_elimination(a,b,d,x,NDIM)

  ! Output Data
  write(*,*)
  write(*,*) "=== Gaussian Elimination ==="
  call sub_output(a,b,d,x,NDIM)

  stop


contains

  subroutine sub_init(a,b,d,x,NDIM,IDEBUG)
    integer, intent(in)             :: NDIM, IDEBUG
    double precision, intent(inout) :: a(NDIM,NDIM), b(NDIM), d(NDIM,NDIM), x(NDIM)
    double precision                :: tmp
    integer                         :: nseed
    integer, allocatable            :: seed(:)

    ! Initialize Puseudo Random Number
    call random_seed(size=nseed)
    allocate(seed(nseed))
    seed(:) = 2
    call random_seed(put=seed)

    ! Initialization
    d(:,:) = 0.0
    do j = 1, NDIM
      do i = 1, NDIM
         call random_number(tmp)
         if(i == j) then
           a(j,i) = dble(int(10.0*tmp))
           d(j,i) = 1.0
         else
           a(j,i) = 1.0
         endif
      enddo
      call random_number(tmp)
      b(j) = dble(int(10.0*tmp))
      x(j) = 0.0
    enddo

    deallocate(seed)

    if(IDEBUG == 1 .and. NDIM == 3) then
      a(1,1) = 2.0
      a(1,2) = 1.0
      a(1,3) = 2.0
      a(2,1) = 1.0
      a(2,2) = 3.0
      a(2,3) = 1.0
      a(3,1) = 1.0
      a(3,2) = 1.0
      a(3,3) = 3.0
      b(1) = 5.0
      b(2) = 0.0
      b(3) = 6.0
    endif

    return
  end subroutine sub_init


  subroutine sub_gaussian_elimination(a,b,d,x,NDIM)
    integer, intent(in)             :: NDIM
    double precision, intent(inout) :: a(NDIM,NDIM), b(NDIM), d(NDIM,NDIM), x(NDIM)
    integer                         :: i, j, k
    double precision                :: c, dm, tmp

    ! Forward Elimination
    do k = 1, NDIM-1
      ! Avoidance of Zero Divide (Swap of Rows)
      call sub_pivot(a,b,d,x,k,NDIM)

      ! Search rows (Skip i = k)
      do i = k+1, NDIM
        c = a(i,k)/a(k,k)
        write(*,*) "a(",i,",",k,")=",a(i,k), "a(",k,",",k,")=",a(k,k)
        a(i,k) = 0.0
        d(i,k) = c
        ! Search columns
        do j = k+1, NDIM
          a(i,j) = a(i,j) - c*a(k,j)
        enddo
        b(i) = b(i) - c*b(k)
      enddo
    enddo

    ! Backward Substitution
    x(NDIM) = b(NDIM)/a(NDIM,NDIM)
    do k = NDIM-1, 1, -1
      tmp = 0.0
      do j = k+1, NDIM
        tmp = tmp + a(k,j)*x(j)
      enddo
      x(k) = (b(k) - tmp)/a(k,k)
    enddo

    return
  end subroutine sub_gaussian_elimination


  subroutine sub_pivot(a,b,d,x,n,NDIM)
    integer, intent(in)             :: n, NDIM
    double precision, intent(inout) :: a(NDIM,NDIM), b(NDIM), d(NDIM,NDIM), x(NDIM)
    double precision                :: epsmac, bu_a(NDIM), bu_b, bu_d(NDIM), bu_x
    integer                         :: i, j, itmp

    ! machine epsilon
    epsmac = epsilon(epsmac)

    if(abs(a(n,n)) <= epsmac) then
      j = n
      do j = n+1, NDIM
        if(abs(a(j,n)) > epsmac) then
          ! Exchange rows n->j, j->n
          ! Backup
          bu_a(:) = a(j,:)
          bu_d(:) = d(j,:)
          bu_b    = b(j)
          bu_x    = x(j)

          ! Substitute n->j
          a(j,:)  = a(n,:)
          d(j,:)  = d(n,:)
          b(j)    = b(n)
          x(j)    = x(n)

          ! Substitute j->n
          a(n,:)  = bu_a(:)
          d(n,:)  = bu_d(:)
          b(n)    = bu_b
          x(n)    = bu_x
          exit
        endif
      enddo
      if(j == n) then
        write(*,*) "[ERROR] Pivotting Failed. "
      endif
    endif

    return
  end subroutine sub_pivot


  subroutine sub_output(a,b,d,x,NDIM)
    integer, intent(in)             :: NDIM
    double precision, intent(inout) :: a(NDIM,NDIM), b(NDIM), d(NDIM,NDIM), x(NDIM)

    write(*,*) "L="
    do j = 1, NDIM
      do i = 1, NDIM
        write(*,'(1f5.1)',advance='no') d(j,i)
      enddo
      write(*,*) ""
    enddo

    write(*,*) "A="
    do j = 1, NDIM
      do i = 1, NDIM
        write(*,'(1f5.1)',advance='no') a(j,i)
      enddo
      write(*,*) ""
    enddo

    write(*,*) "B="
    do j = 1, NDIM
      write(*,'(1f5.1)') b(j)
    enddo

    write(*,*) "X="
    do j = 1, NDIM
      write(*,'(1f5.1)') x(j)
    enddo

    return
  end subroutine sub_output

end program main

Writing ACS7.f03


In [3]:
DEBUG=0
EXE="ACS7"
if DEBUG == 0:
  !gfortran {EXE}.f03 -o ./{EXE}.out
  !./{EXE}.out
else:
  !sudo apt install gdb
  !gfortran {EXE}.f03 -o ./{EXE}.out -g
  !echo "run" | gdb ./{EXE}.out

 === Initial Value ===
 L=
  1.0  0.0  0.0 
  0.0  1.0  0.0 
  0.0  0.0  1.0 
 A=
  2.0  1.0  2.0 
  1.0  3.0  1.0 
  1.0  1.0  3.0 
 B=
  5.0
  0.0
  6.0
 X=
  0.0
  0.0
  0.0
 a(           2 ,           1 )=   1.0000000000000000      a(           1 ,           1 )=   2.0000000000000000     
 a(           3 ,           1 )=   1.0000000000000000      a(           1 ,           1 )=   2.0000000000000000     
 a(           3 ,           2 )=  0.50000000000000000      a(           2 ,           2 )=   2.5000000000000000     

 === Gaussian Elimination ===
 L=
  1.0  0.0  0.0 
  0.5  1.0  0.0 
  0.5  0.2  1.0 
 A=
  2.0  1.0  2.0 
  0.0  2.5  0.0 
  0.0  0.0  2.0 
 B=
  5.0
 -2.5
  4.0
 X=
  1.0
 -1.0
  2.0
